## Custom Maxwell Solvers
Our FDFD Maxwell solver is great for simple tasks. The solver does not need to be fast: to compute limits, you only need to compute the inverse of the background Green's function mapping currents in the design region to fields in the design region, *once*. 

Still, if you want to slot in your own Maxwell solver, this tutorial will cover how to do so. This might help have better numerical consistency between the limits and your own inverse designs, or allow you to calculate limits for more complex systems.

In [1]:
# %autoreload 2 # Autoreloading will not work in this file because of inspect. Restart the kernel if you need to reload. 
import inspect
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sys, time, os

package_path = os.path.abspath('../../dolphindes')
if package_path not in sys.path:
    sys.path.append(package_path)

from dolphindes import photonics, maxwell

In [2]:
# The photonics class has an attribute, self.EM_solver, which it calls to compute electromagnetic quantities. This is calculated in setup_EM_solver():
print(inspect.getsource(photonics.Photonics_TM_FDFD.setup_EM_solver))

    def setup_EM_solver(self, omega=None, Nx=None, Ny=None, Npmlx=None, Npmly=None, dl=None, bloch_x=None, bloch_y=None):
        """
        setup the solver. non-None arguments will define / modify corresponding attributes
        """
        params = locals()
        params.pop('self')
        for param_name, param_value in params.items():
            if param_value is not None:
                setattr(self, param_name, param_value)

        check_attributes(self, 'Nx', 'Ny', 'Npmlx', 'Npmly', 'dl', 'bloch_x', 'bloch_y')
        self.EM_solver = TM_FDFD(self.omega, self.Nx, self.Ny, self.Npmlx, self.Npmly, self.dl, self.bloch_x, self.bloch_y)



In [3]:
# TM_FDFD is our TM FDFD code. All you have to do is over-write this function with your own custom solver. i.e. 

class custom_maxwell_photonics(photonics.Photonics_TM_FDFD):
    def setup_EM_solver(self, omega, Nx, Ny, Npmlx, Npmly, dl, bloch_x, bloch_y):
        # Do whatever you want with the parameters 
        self.EM_solver = None # here replace your custom solver
        pass 

In [ ]:
# What should the EM_solver know what to do? 
# At minimum, it should be able to compute the off-diagonal block of the inverse of the Green's function which corresponds to design -> design. This gets called in setup_EM_operators, which is needed for setting up the QCQP for limit calculations. 
# (This may look different once everything has been implemented)
print(inspect.getsource(photonics.Photonics_TM_FDFD.setup_EM_operators))

    def setup_EM_operators(self):
        """
        setup EM operators associated with the given design region and background
        """
        check_attributes(self, 'des_mask')
        if self.sparseQCQP:
            self.Ginv, self.M = self.EM_solver.get_GaaInv(self.des_mask, self.chi_background)
        else:
            raise ValueError("dense QCQP not implemented yet")



In [8]:
# For reference, this is what our get_GaaInv looks like:
print(inspect.getsource(maxwell.TM_FDFD.get_GaaInv))

    def get_GaaInv(self, A_mask: np.ndarray, chigrid: np.ndarray = None) -> tuple[np.ndarray, sp.csc_array]:
        """
        Compute the inverse Green’s function on region A, G_{AA}^{-1}, using a Woodbury identity.

        We partition the full Maxwell operator M into blocks corresponding to region A (design)
        and its complement B (background):
            M = [[A, B],
                 [C, D]]
        Then G_{AA}^{-1} = D - C A^{-1} B, up to a multiplicative constant MU_0 / k^2.

        Parameters
        ----------
        A_mask : np.ndarray of bool, shape (Nx, Ny)
            Mask for the design region A.
        chigrid : np.ndarray of complex, optional
            Material susceptibility distribution. If provided, M = M0 + diag(ω² χ).

        Returns
        -------
        GaaInv : sp.csc_array of shape (n_A, n_A)
            The inverse Green’s function on region A.
        M : sp.csc_array
            The full Maxwell operator used in the computation.
        """


In [4]:
# What else should the EM_solver know what to do? If you plan to pass an incident current, it needs to know how to calculate fields from the currents. This is called in get_ei()
print(inspect.getsource(photonics.Photonics_TM_FDFD.get_ei))
# Alternatively, you can just pass ji = None and set_ei with a known incident field of size Nx, Ny via the attribute self.ei or by calling set_ei()

    def get_ei(self, ji = None, update=False):
        """
        get the incident field
        """
        if self.ei is None:
            ei = self.EM_solver.get_TM_field(ji, self.chi_background) if self.ji is None else self.EM_solver.get_TM_field(self.ji, self.chi_background)
        else:
            ei = self.ei        
        if update: self.ei = ei
        return ei

